In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
bureau = pd.read_csv(PATH + 'bureau.csv')
bb = pd.read_csv(PATH + 'bureau_balance.csv')
b = bureau.copy()
'done'
df = pd.concat([train[['SK_ID_CURR']], test[['SK_ID_CURR']]], axis = 0)
print('bshape',b.shape)
print('dfshaoe',df.shape)

bshape (1716428, 17)
dfshaoe (356255, 1)


In [2]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def multi_name(col1, col2): return col1 + '_times_' + col2
def multi(df, col1, col2): return df[col1] * df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [3]:
df = pd.DataFrame(bureau['SK_ID_BUREAU'])
merged_col = 'SK_ID_BUREAU'

smap = {'C':0, 'X':0, '0':0, '1':1, '2':2, '3':3, '4':4, '5':5}
bb['STATUS_NUM'] = bb['STATUS'].map(smap)
# ratio
bb[ratio_name('STATUS_NUM', 'MONTHS_BALANCE')] = ratio(bb, 'STATUS_NUM', 'MONTHS_BALANCE')

# one hot sum
df = df.merge(one_hot(bb, merged_col, 'STATUS'), on=merged_col, how='left')

ignored = ['SK_ID_BUREAU', 'STATUS', 'STATUS_NUM']
for x in set(bb.columns) - set(ignored):
    print(x)
    df = df.merge(numerical(bb, merged_col, x, ['mean', 'median', 'max', 'min', 'sum']), on=merged_col, how='left')

'done'

MONTHS_BALANCE
STATUS_NUM_divide_MONTHS_BALANCE


'done'